In [1]:
import os
import sys

import constants
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain


os.environ['OPENAI_API_KEY'] = constants.APIKEY
repo_path = "../ProjectAeon/aeon_mecha"

loader = GenericLoader.from_filesystem(
    repo_path+"/aeon",
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)
documents = loader.load()

python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, 
                                                               chunk_size=2000, 
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)

db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

llm = ChatOpenAI(model_name="gpt-4") 
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [4]:
question = "What is the 'aeon.api.load' function doing?"
result = qa(question)
print(result['answer'])

The 'aeon.api.load' function is not defined in the provided context, so I cannot provide details on what it does.


In [6]:
question = "What does Aeon's api.load function do?"
result = qa(question)
print(result['answer'])

The `api.load` function in Aeon's API is used for reading data from specified raw files in an Aeon dataset. It requires the `data_dir`, `reader`, `start` and `end` as parameters. The `data_dir` specifies the directory where the data files are located. The `reader` parameter is an instance of the `Reader` class, which is used to specify the pattern, columns, and extension of the data file. The `start` and `end` parameters specify the range of data to be read from the files. Please note that the actual implementation of the function is not provided in the given context.


In [3]:
question = "What do the Experiment tables in the database contain?"
result = qa(question)
print(result['answer'])

The Experiment table in the database contains information about a particular experiment. Here are the fields it contains:

- "experiment_name": A string (varchar) of up to 32 characters representing the name of the experiment, for example "exp0-r0".

- "experiment_start_time": A timestamp (datetime) indicating when the experiment started.

- "experiment_description": A string (varchar) of up to 1000 characters providing a description of the experiment.

- "lab.Arena": A foreign key (->) that links to the Arena table in the lab schema. It represents the arena where the experiment took place.

- "lab.Location": A foreign key (->) that links to the Location table in the lab schema. It indicates the lab or room where the experiment took place.

- "ExperimentType": A foreign key (->) that links to the ExperimentType table. It denotes the type of the experiment conducted.

The Experiment table also has two subclasses: Subject and Directory.

- The Subject subclass contains a list of subjects

In [5]:
question = "How could I create a new Device made out of streams from patches and cameras?"
result = qa(question)
print(result['answer'])

To create a new device using streams from patches and cameras, you can follow the code structure provided in the context. This involves two main steps:

1. Defining the properties and behavior of the new device: This is done using a combination of Python classes and functions. In the provided code, this is done with the function `generate_device_table(device_type)`. This function generates a device table with specific properties like experiment device, installation time, description, and removal time. It also generates a data stream table for the device.

2. Adding the device to an experiment setup: After defining the device and its behavior, the new device needs to be added to an experiment setup. This is done by creating instances of the new device and adding them to a DotMap. In the provided code, this is done in the `exp01`, `exp02` and `octagon01` setups.

For instance, if you want to create a new camera device, you could follow the structure used to define the `CameraTop` device 

In [7]:
questions = [
    "What is the 'aeon.api.load' function doing?",
    "What do the Experiment tables in the database contain?",
    "How could I create a new Device made out of streams from patches and cameras?",
]

for question in questions:
    result = qa(question)
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: What is the 'aeon.api.load' function doing? 

**Answer**: The 'aeon.api.load' function is not explicitly defined in the provided context. However, based on its usage in the '_load_legacy_subjectdata' function, it appears to be a function used for loading data from a specific location (specified by 'data_dir') using a specific reader (specified by 'reader'), within a certain time frame (specified by 'start' and 'end'). It seems to be part of Aeon's API for handling and loading data. 

-> **Question**: What do the Experiment tables in the database contain? 

**Answer**: The Experiment table in the database contains the following information:

1. `experiment_name`: A string that represents the name of the experiment. For example, 'exp0-r0'.

2. `experiment_start_time`: The date and time when the experiment started.

3. `experiment_description`: A text description of the experiment.

4. `lab.Arena`: A reference to the Arena table in the lab schema, which contains details a